# Training a *1-layer SoftMax classifier* on the *MNIST* dataset

In this brief experiment, we will test the accuracy achieved by a simple *logistic regression model* (a.k.a. *SoftMax classifier*) on the MNIST dataset. Layer weights are determined via *vanilla-SGD*-optimized error backpropagation.  
Given such tight bounds, all remaining hyperparameters can be tuned as to maximize accuracy.

In [1]:
# Data download:
import os

# NNets & co.:
import numpy as np
import torch as th
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

# Data(set) handling
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor, Normalize, Compose, Lambda


In [2]:
# MNIST DataLoader(s) builder


def spawn_mnist_loaders(
    data_root="datasets/",
    batch_size_train=256,
    batch_size_test=512,
    cuda_accel=False,
    **kwargs
):

    os.makedirs(data_root, exist_ok=True)

    transforms = Compose(
        [
            ToTensor(),
            Normalize((0.1307,), (0.3081,)),  # usual magic constants for MNIST
            Lambda(lambda x: th.flatten(x)),
        ]
    )

    trainset = MNIST(data_root, train=True, transform=transforms, download=True)
    testset = MNIST(data_root, train=False, transform=transforms, download=True)

    cuda_args = {}
    if cuda_accel:
        cuda_args = {"num_workers": 1, "pin_memory": True}

    trainloader = DataLoader(
        trainset, batch_size=batch_size_train, shuffle=True, **cuda_args
    )
    testloader = DataLoader(
        trainset, batch_size=batch_size_test, shuffle=False, **cuda_args
    )

    return trainloader, testloader


In [3]:
# Parameterized SoftMax Classifier


class SMC(th.nn.Module):
    def __init__(self, fin: int, fout: int):
        super().__init__()
        self.layer1 = th.nn.Linear(in_features=fin, out_features=fout)

    def forward(self, X):
        out = self.layer1(X)
        out = F.log_softmax(out, dim=1)
        return out


In [4]:
# Train / Test tooling


def train_epoch(
    model, device, train_loader, loss_fn, optimizer, epoch, print_every_nep
):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % print_every_nep == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                )
            )


def test(model, device, test_loader, loss_fn):
    model.eval()
    test_loss = 0
    correct = 0
    with th.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(
                output, target, reduction="sum"
            ).item()  # sum up batch loss
            pred = output.argmax(
                dim=1, keepdim=True
            )  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss,
            correct,
            len(test_loader.dataset),
            100.0 * correct / len(test_loader.dataset),
        )
    )


In [5]:
device = th.device("cuda" if th.cuda.is_available() else "cpu")


In [6]:
# Hyperparameters & co.

minibatch_size_train: int = 32  # (cfr. Masters & Luschi, 2018)
minibatch_size_test: int = 512

nrepochs = 20

lossfn = F.nll_loss


In [7]:
train_loader, test_loader = spawn_mnist_loaders(
    batch_size_train=minibatch_size_test,
    batch_size_test=minibatch_size_test,
    cuda_accel=True,
)


In [8]:
model = SMC(28 * 28, 10).to(device)
optimizer = th.optim.SGD(model.parameters(), lr=0.7)
scheduler = StepLR(optimizer, step_size=1, gamma=0.75)  # or a lot of patience :)


In [9]:
for epoch in range(1, nrepochs + 1):
    train_epoch(
        model, device, train_loader, lossfn, optimizer, epoch, print_every_nep=10
    )
    test(model, device, test_loader, lossfn)
    test(model, device, test_loader, lossfn)
    scheduler.step()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.408727
Train Epoch: 1 [5120/60000 (8%)]	Loss: 1.599636
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.097477
Train Epoch: 1 [15360/60000 (25%)]	Loss: 1.587394
Train Epoch: 1 [20480/60000 (34%)]	Loss: 1.427219
Train Epoch: 1 [25600/60000 (42%)]	Loss: 0.690546
Train Epoch: 1 [30720/60000 (51%)]	Loss: 3.272240
Train Epoch: 1 [35840/60000 (59%)]	Loss: 0.916544
Train Epoch: 1 [40960/60000 (68%)]	Loss: 2.618241
Train Epoch: 1 [46080/60000 (76%)]	Loss: 0.598979
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.626734
Train Epoch: 1 [56320/60000 (93%)]	Loss: 0.543361

Test set: Average loss: 2.1848, Accuracy: 45735/60000 (76%)


Test set: Average loss: 2.1848, Accuracy: 45735/60000 (76%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.336122
Train Epoch: 2 [5120/60000 (8%)]	Loss: 0.508835
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.386824
Train Epoch: 2 [15360/60000 (25%)]	Loss: 0.520472
Train Epoch: 2 [20480/60000 (34%)]	Loss: 0.927284
Train Epoch: 2 [25600/60000 (42%)]	Loss: